# __Demo: Creating a Custom Template__



## __Steps to Perform__
Step 1: Set up the Environment

Step 2: Define the Prompt Template

Step 3: Create a Custom Prompt Template Class



### __Step 1: Set up the Environment__

- Import required modules such as StringPromptTemplate from the langchain.prompts library and BaseModel and validator from Pydantic library.



In [ ]:
age='25'

In [15]:
from pydantic import BaseModel, ValidationError, Field
class User(BaseModel):
    id: int
    name: str= Field(min_length=3)
    email:str
    
u1 = User(id='1', name="ra", email="a@gmail.com")
u1

ValidationError: 1 validation error for User
name
  String should have at least 3 characters [type=string_too_short, input_value='ra', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short

In [6]:
try:
    u2 = User(id='1', name="raj")
except ValidationError as e:
    print(e)

1 validation error for User
email
  Field required [type=missing, input_value={'id': '1', 'name': 'raj'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing


In [7]:
try:
    u2 = User(id='abc', name="raj", email="not-email")
except ValidationError as e:
    print(e)

1 validation error for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing


In [ ]:
# Serilization:object----->json
# DeSerilization: json---python object

In [11]:
u1.model_dump_json()

'{"id":1,"name":"raj","email":"a@gmail.com"}'

In [12]:
User.model_validate_json('{"id":1,"name":"raj","email":"a@gmail.com"}')

User(id=1, name='raj', email='a@gmail.com')

In [16]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator, ValidationError
from typing import List
import openai

### __Step 2: Define the Prompt Template__
- Define a constant string PROMPT that outlines the structure of the prompt to be used for generating book summaries.
- Include placeholders for dynamic content like the book title.

In [17]:
PROMPT = """\
Given the book title, generate a brief summary of the book.
Book Title: {book_title}
Summary:
"""


### __Step 3: Create a Custom Prompt Template Class__

- The custom class BookSummarizerPrompt is created, which does not inherit from __StringPromptTemplate__ as initially described.
- It has a template field of type __StringPromptTemplate__. This class is responsible for integrating the prompt template with dynamic content (the book title) and is built on BaseModel from Pydantic to enable input validation.
- The Config class within __BookSummarizerPrompt__ sets __arbitrary_types_allowed = True__, allowing for the inclusion of custom types like __StringPromptTemplate__ as a field in the Pydantic model.
- A __create_prompt__ method is implemented to dynamically insert the book title into the prompt template.

**Pydantic** is a Python library that helps check and organize data. It ensures the data is correct and in the right format, automatically fixing types if needed. It is useful for validating user input, API requests, and managing settings easily. 

In [21]:
# let's first understand how Pydantic work
# Define a Pydantic model
class User(BaseModel):
    id: int
    name: str
    email: str
    age: int
    friends: List[str] = []

# # Valid data
data = {
    "id": 123,
    "name": "Alice",
    "email": "alice@example.com",
    "age": 25,
    "friends": ["Raghav", "Afaque", "Ankit"]
}

# # Instantiate the model with valid data
user = User(**data)
print(user)

# Invalid data (age should be an integer)
invalid_data = {
    "id": 124,
    "name": "Bob",
    "email": "bob@example.com",
    "age": "twenty-five",  # Invalid age
    "friends": [234, 345]
}

# user = User(**invalid_data)
# print(user)
try:
    invalid_user = User(**invalid_data)
except ValidationError as e:
    print(e.json())


id=123 name='Alice' email='alice@example.com' age=25 friends=['Raghav', 'Afaque', 'Ankit']
[{"type":"int_parsing","loc":["age"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"twenty-five","url":"https://errors.pydantic.dev/2.10/v/int_parsing"},{"type":"string_type","loc":["friends",0],"msg":"Input should be a valid string","input":234,"url":"https://errors.pydantic.dev/2.10/v/string_type"},{"type":"string_type","loc":["friends",1],"msg":"Input should be a valid string","input":345,"url":"https://errors.pydantic.dev/2.10/v/string_type"}]


In [ ]:
# Define a Pydantic model
class User(BaseModel):
    id: int
    name: str
    email: str
    age: int
    friends: List[str] = []

    class Config:
        strict = True
# # # Valid data
# data = {
#     "id": 123,
#     "name": "Alice",
#     "email": "alice@example.com",
#     "age": 25,
#     "friends": ["Raghav", "Afaque", "Ankit"]
# }

# # # Instantiate the model with valid data
# # user = User(**data)
# # print(user)

# Invalid data (age should be an integer)
invalid_data = {
    "id": 124,
    "name": "Bob",
    "email": "bob@example.com",
    "age": 25,  # Invalid age
    "friends": [234, 345]
    # "friends": ["Raghav", "Afaque", "Ankit"]
}

user = User(**invalid_data)
print(user)
# try:
#     invalid_user = User(**invalid_data)
# except ValidationError as e:
#     print(e.json())


In [24]:
#encaps, reuse
class StringPromptTemplate:
    def __init__(self, template: str):
        self.template = template

    def format(self, **kwargs) -> str:
        return self.template.format(**kwargs)

class BookSummarizerPrompt(BaseModel):
    book_title: str
    template: StringPromptTemplate

    class Config:
        arbitrary_types_allowed = True

    def create_prompt(self) -> str:
        return self.template.format(book_title=self.book_title)

# Define the template
# summary_template = StringPromptTemplate(
#     """
# Given the book title, generate a brief summary of the book.
# Book Title: {book_title}
# Summary:
# """
# )
template = StringPromptTemplate(
    """
Given the book title, generate a brief review of the book.
Book Title: {book_title}
review:
"""
)
# Create instance of BookSummarizerPrompt
book_summarizer_prompt = BookSummarizerPrompt(book_title="The Great Gatsby", template=template)

# Generate prompt
formatted_prompt = book_summarizer_prompt.create_prompt()
print("Generated Prompt:")
print(formatted_prompt)

# Define OpenAI completion function
def get_completion(prompt: str, model="gpt-3.5-turbo") -> str:
    try:
        messages = [{"role": "user", "content": prompt}]
        
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=0.1,
            top_p=0.8,
            max_tokens=512
        )

        return response["choices"][0]["message"]["content"]

    except openai.error.OpenAIError as e:
        return f"OpenAI API Error: {str(e)}"
    except Exception as e:
        return f"Unexpected Error: {str(e)}"

# Get AI-generated summary
response = get_completion(formatted_prompt)
print("\nAI Response:")
print(response)


Generated Prompt:

Given the book title, generate a brief review of the book.
Book Title: The Great Gatsby
review:


AI Response:
"The Great Gatsby" is a timeless classic that delves into the decadence and excess of the Jazz Age. F. Scott Fitzgerald's writing is both poetic and poignant, capturing the essence of the American Dream and the emptiness that can come with it. The characters are complex and flawed, making the story both captivating and tragic. Overall, "The Great Gatsby" is a must-read for anyone looking for a beautifully crafted novel that explores themes of love, wealth, and the pursuit of happiness.


In [26]:
BookSummarizerPrompt(book_title="1984", template=review_template)

BookSummarizerPrompt(book_title='1984', template=<__main__.StringPromptTemplate object at 0x7f81c0d17b20>)

In [27]:
# Define a template class to manage dynamic text
class RecipePromptTemplate:
    def __init__(self, template: str):
        self.template = template

    def format(self, **kwargs):
        return self.template.format(**kwargs)

# Define a pydantic model for recipe inputs
class RecipeGenerator(BaseModel):
    recipe_name: str
    ingredients: List[str]
    template: RecipePromptTemplate

    def create_prompt(self):
        # Join ingredients into a single string for the template
        ingredients_text = ", ".join(self.ingredients)
        return self.template.format(recipe_name=self.recipe_name, ingredients=ingredients_text)

    class Config:
        arbitrary_types_allowed = True

# Define the prompt template
recipe_template = RecipePromptTemplate(
    """
    Generate a detailed recipe for the following dish:
    Recipe Name: {recipe_name}
    Ingredients: {ingredients}
    Instructions:
    """
)

# Create an instance of the recipe generator
recipe_data = {
    "recipe_name": "Pizza",
    "ingredients": ["Refined Flour", "eggs", "parmesan cheese", "Mushrooms", "black pepper"],
    "template": recipe_template,
}

recipe_generator = RecipeGenerator(**recipe_data)

# Generate the prompt
prompt = recipe_generator.create_prompt()
print("Prompt:")
print(prompt)
response = get_completion(recipe_generator.create_prompt())
print("AI Response:")
print(response)

Prompt:

    Generate a detailed recipe for the following dish:
    Recipe Name: Pizza
    Ingredients: Refined Flour, eggs, parmesan cheese, Mushrooms, black pepper
    Instructions:
    
AI Response:
1. Preheat your oven to 450°F (230°C).
2. In a mixing bowl, combine 2 cups of refined flour, 2 eggs, and a pinch of salt. Mix well until a dough forms.
3. On a floured surface, knead the dough for about 5 minutes until it becomes smooth and elastic.
4. Roll out the dough into a round shape, about 12 inches in diameter, and place it on a greased pizza pan.
5. Spread a layer of tomato sauce over the dough, leaving a small border around the edges.
6. Sprinkle a generous amount of grated parmesan cheese over the sauce.
7. Slice some mushrooms and arrange them on top of the cheese.
8. Season with black pepper to taste.
9. Bake the pizza in the preheated oven for 15-20 minutes, or until the crust is golden brown and the cheese is melted and bubbly.
10. Remove the pizza from the oven and let it

## __Conclusion__
This demo is designed to dynamically generate book summaries and recipies by inserting book titles and ingredients into a structured prompt template.

It achieves this through a custom implementation involving a template class for formatting and a Pydantic model for data validation and dynamic content integration, ensuring the output is tailored and accurate.